In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
t_data=pd.read_excel('C:\\Users\\a3011\\OneDrive\\Desktop\\사회조사분석(특성항목)\\2020년 경상남도 사회조사 시군 특성항목 조사자료(수정)\\13. 2020년 경상남도 사회조사 고성군 특성항목 조사자료.xlsx')

In [3]:
sakin=pd.read_excel('C:\\Users\\a3011\\OneDrive\\Desktop\\사회조사분석(특성항목)\\2020년 경상남도 사회조사 시군 특성항목 조사자료(수정)\\고성군색인.xlsx')

In [4]:
# 나이 분류
age=t_data['가구원_만나이']
age[age<=19]=1
age[(20<=age) &(age<=29)]=2
age[(30<=age) &(age<=39)]=3
age[(40<=age) &(age<=49)]=4
age[(50<=age) &(age<=59)]=5
age[(60<=age) &(age<=64)]=6
age[65<=age]=7
age
t_data['나이']=age

In [5]:
#학력별
school=t_data[['일반사항_6.응답자_교육정도','일반사항_6-1.응답자_교육정도_수학여부']]

no_graduate=school[school['일반사항_6-1.응답자_교육정도_수학여부']!=1]
graduate=school[school['일반사항_6-1.응답자_교육정도_수학여부']==1]
no_graduate2=no_graduate['일반사항_6.응답자_교육정도']-1

school_2=pd.concat([graduate['일반사항_6.응답자_교육정도'],no_graduate2]).sort_index()
school_2=pd.DataFrame(school_2)
school_2.columns=['교육정도수준']
school_2[school_2['교육정도수준']<=0]=1
school_2[school_2['교육정도수준']>4]=4
t_data['교육정도수준']=school_2

In [6]:
t_data['교육정도수준'].unique()

array([4, 1, 2, 3], dtype=int64)

In [7]:
#가구 가중치
gagu_weight=t_data.shape[0]*[36.4967]
t_data['가구 가중치']=gagu_weight

In [8]:
#가구원 가중치
gaguwon_index=t_data[['나이','가구원_성별']]
weight=[(1,1,236.0000),(1,2,236.0000),(1,3,69.2188),(1,4,64.0175),(1,5,67.4189),(1,6,34.7977),(1,7,31.6146),
       (2,1,113.9565),(2,2,113.9565),(2,3,44.4878),(2,4,52.2545),(2,5,61.2319),(2,6,30.5216),(2,7,28.6918)]
for i,j,k in weight:
    gaguwon_index[(gaguwon_index['나이']==j) & (gaguwon_index['가구원_성별']==i)]=k
t_data['가구원 가중치']=gaguwon_index['나이']

In [9]:
t_data[['나이','교육정도수준','가구 가중치','가구원 가중치']]

,나이,교육정도수준,가구 가중치,가구원 가중치
0,4,4,36.4967,52.2545
1,7,1,36.4967,28.6918
2,7,1,36.4967,28.6918
3,7,2,36.4967,31.6146
4,7,1,36.4967,31.6146
...,...,...,...,...
1016,7,2,36.4967,28.6918
1017,7,4,36.4967,28.6918
1018,6,3,36.4967,30.5216
1019,7,4,36.4967,28.6918


In [10]:
# 가구원 가중치 데이터
df_index = ['성별','남자','여자','나이','15~19','20~29','30~39','40~49','50~59','60~64','65~','학력','초졸이하','중졸','고졸','대졸이상',
           '거처의종류','단독주택','아파트','연립주택','다세대주택','기타','월평균가구소득','~50','50~','100~','200~','300~','400~','500~','600~','700~',
          '800~','주거점유형태','자기집','전세','보증금있는월세','보증금없는월세','무상']

a = pd.DataFrame(index=range(1,25),columns=df_index)

row_names = ['남자','여자','15~19','20~29','30~39','40~49','50~59','60~64','65~','초졸이하','중졸','고졸','대졸이상',
           '단독주택','아파트','연립주택','다세대주택','기타','~50','50~','100~','200~','300~','400~','500~','600~','700~',
          '800~','자기집','전세','보증금있는월세','보증금없는월세','무상']

identity_category = ['가구원_성별','나이','교육정도수준','일반사항_1.응답자_거처의종류','17-1.가구_월평균가구소득','일반사항_2.응답자_주거점유형태']

sakin2 = sakin

gaguwon_sakin = sakin2[sakin['가중치']=='가구원 가중치']

spl_data = t_data[identity_category]

for i in gaguwon_sakin['변수명']:
    
    spl_df = a.copy()
    if (i in spl_data) == True:
        using_data = spl_data
        using_data = pd.concat([using_data,t_data['가구원 가중치']],axis=1)
    
    else:
        using_data = pd.concat([spl_data,t_data[i]],axis=1)
        using_data = pd.concat([using_data,t_data['가구원 가중치']],axis=1)
    for j in identity_category:
        
        if j=='가구원_성별':
            for idx,k in [('남자',1),('여자',2)]:
                using_data2=using_data[using_data[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        elif j=='나이':
            for idx,k in [('15~19',1),('20~29',2),('30~39',3),('40~49',4),('50~59',5),('60~64',6),('65~',7)]:
                using_data2=using_data[using_data[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        elif j=='교육정도수준':
            for idx,k in [('초졸이하',1),('중졸',2),('고졸',3),('대졸이상',4)]:
                using_data2=using_data[using_data[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        elif j=='일반사항_1.응답자_거처의종류':
            for idx,k in [('단독주택',1),('아파트',2),('연립주택',3),('다세대주택',4),('기타',5)]:
                using_data2=using_data[using_data[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        elif j=='17-1.가구_월평균가구소득':
            for idx,k in [('~50',1),('50~',2),('100~',3),('200~',4),('300~',5),('400~',6),('500~',7),('600~',8),('700~',9),('800~',10)]:
                using_data2=using_data[using_data[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        elif j=='일반사항_2.응답자_주거점유형태':
            for idx,k in [('자기집',1),('전세',2),('보증금있는월세',3),('보증금없는월세',4),('무상',5)]:
                using_data2=using_data[using_data[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        


    spl_df=spl_df.transpose()
    spl_df.to_excel('C:\\Users\\a3011\\OneDrive\\Desktop\\사회조사분석(특성항목)\\특성항목엑셀\\'+'고성군'+str(i)+'.xlsx')

In [11]:
# 통합데이터 사용
for m in ['통합데이터1']:
    row_names= ['남자','여자','15~19','20~29','30~39','40~49','50~59','60~64','65~','초졸이하','중졸','고졸','대졸이상',
           '단독주택','아파트','연립주택','다세대주택','기타','~50','50~','100~','200~','300~','400~','500~','600~','700~',
          '800~','자기집','전세','보증금있는월세','보증금없는월세','무상']
    df_index=['성별','남자','여자','나이','15~19','20~29','30~39','40~49','50~59','60~64','65~','학력','초졸이하','중졸','고졸','대졸이상',
           '거처의종류','단독주택','아파트','연립주택','다세대주택','기타','월평균가구소득','~50','50~','100~','200~','300~','400~','500~','600~','700~',
          '800~','주거점유형태','자기집','전세','보증금있는월세','보증금없는월세','무상']
    a=pd.DataFrame(index=range(1,25),columns=df_index)
    identity_category=['가구원_성별','나이','교육정도수준','일반사항_1.응답자_거처의종류','17-1.가구_월평균가구소득','일반사항_2.응답자_주거점유형태']
    sakin3=sakin[sakin['가중치']=='가구원 가중치']
    sakin2=sakin3[sakin3['데이터여부']==m]
    save_df=pd.DataFrame()
    for i in sakin2['변수명']:
        abcd=t_data[[i,'가구원 가중치','가구원_성별','나이','교육정도수준','일반사항_1.응답자_거처의종류','17-1.가구_월평균가구소득','일반사항_2.응답자_주거점유형태']]
        
        abcd.columns=['통합정리','가구원 가중치','가구원_성별','나이','교육정도수준','일반사항_1.응답자_거처의종류','17-1.가구_월평균가구소득','일반사항_2.응답자_주거점유형태']
        
        save_df=pd.concat([save_df,abcd])
        
    save_df=save_df[save_df['통합정리'].isnull()==False]
    i='통합정리'
    spl_df=a.copy()
    
    for j in identity_category:
        
        if j=='가구원_성별':
            for idx,k in [('남자',1),('여자',2)]:
                using_data2=save_df[save_df[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        elif j=='나이':
            for idx,k in [('15~19',1),('20~29',2),('30~39',3),('40~49',4),('50~59',5),('60~64',6),('65~',7)]:
                using_data2=save_df[save_df[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        elif j=='교육정도수준':
            for idx,k in [('초졸이하',1),('중졸',2),('고졸',3),('대졸이상',4)]:
                using_data2=save_df[save_df[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        elif j=='일반사항_1.응답자_거처의종류':
            for idx,k in [('단독주택',1),('아파트',2),('연립주택',3),('다세대주택',4),('기타',5)]:
                using_data2=save_df[save_df[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        elif j=='17-1.가구_월평균가구소득':
            for idx,k in [('~50',1),('50~',2),('100~',3),('200~',4),('300~',5),('400~',6),('500~',7),('600~',8),('700~',9),('800~',10)]:
                using_data2=save_df[save_df[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
        elif j=='일반사항_2.응답자_주거점유형태':
            for idx,k in [('자기집',1),('전세',2),('보증금있는월세',3),('보증금없는월세',4),('무상',5)]:
                using_data2=save_df[save_df[j] == k]
                data_series=using_data2['가구원 가중치'].groupby(using_data2[i]).sum() / using_data2['가구원 가중치'].groupby(using_data2[i]).sum().sum()*100
                spl_df[idx] = data_series
       
        
            

    spl_df=spl_df.transpose()
    spl_df.to_excel('C:\\Users\\a3011\\OneDrive\\Desktop\\사회조사분석(특성항목)\\특성항목엑셀\\'+'고성군(통합데이터)'+str(m)+'.xlsx') 

KeyError: '통합정리'

In [ ]:
for i in sakin['변수명']:
    aa=t_data[[i,'가구원 가중치']]
    print(aa['가구원 가중치'].groupby(aa[i]).sum()/aa['가구원 가중치'].groupby(aa[i]).sum().sum()*100)